# 🎓 Week 7 - Day 1: Scikit-Learn Advanced Techniques

## Today's Goals:
✅ Learn hyperparameter tuning with GridSearchCV & RandomizedSearchCV

✅ Master cross-validation strategies

✅ Apply PCA for dimensionality reduction

✅ Save and load models properly

---

## Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import (
    train_test_split, GridSearchCV, RandomizedSearchCV,
    cross_val_score, KFold, StratifiedKFold
)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print('✅ Libraries imported!')

✅ Libraries imported!


---
## Part 1: Understanding Hyperparameters

**What are hyperparameters?**
Settings YOU choose before training (not learned from data)

In [3]:
# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

print(f'Training samples: {len(X_train)}')
print(f'Test samples: {len(X_test)}')

Training samples: 120
Test samples: 30


In [4]:
# Example: Random Forest has hyperparameters
print('Random Forest Hyperparameters:')
print('• n_estimators: How many trees? (50, 100, 200...)')
print('• max_depth: How deep each tree? (3, 5, 10...)')
print('• min_samples_split: Min samples to split? (2, 5, 10...)')
print('\nQuestion: Which values work best? 🤔')

Random Forest Hyperparameters:
• n_estimators: How many trees? (50, 100, 200...)
• max_depth: How deep each tree? (3, 5, 10...)
• min_samples_split: Min samples to split? (2, 5, 10...)

Question: Which values work best? 🤔


---
## Part 2: Manual Tuning (The Hard Way)

**Let's try different values manually**

In [5]:
# Try different n_estimators
results = []

for n in [10, 50, 100, 200]:
    model = RandomForestClassifier(n_estimators=n, random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    results.append({'n_estimators': n, 'accuracy': score})
    print(f'n_estimators={n:3d} → Accuracy: {score:.3f}')

print('\n❌ This is tedious! Imagine trying 100s of combinations!')
print('✅ GridSearchCV automates this! ⬇️')

n_estimators= 10 → Accuracy: 1.000
n_estimators= 50 → Accuracy: 1.000
n_estimators=100 → Accuracy: 1.000
n_estimators=200 → Accuracy: 1.000

❌ This is tedious! Imagine trying 100s of combinations!
✅ GridSearchCV automates this! ⬇️


---
## Part 3: GridSearchCV (The Smart Way)

**Automatically tries all combinations!**

In [6]:
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7]
}

print('Parameter Grid:')
print(f"n_estimators: {param_grid['n_estimators']}")
print(f"max_depth: {param_grid['max_depth']}")
print(f'\nTotal combinations to test: 3 × 3 = 9')

Parameter Grid:
n_estimators: [50, 100, 150]
max_depth: [3, 5, 7]

Total combinations to test: 3 × 3 = 9


In [7]:
# Create GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    rf, param_grid, cv=5, scoring='accuracy', verbose=1
)

print('✅ GridSearchCV created!')
print('Will test 9 combinations with 5-fold CV')
print('Total fits: 9 × 5 = 45\n')

# Fit (this tries all combinations!)
grid_search.fit(X_train, y_train)

print('\n✅ Grid search completed!')

✅ GridSearchCV created!
Will test 9 combinations with 5-fold CV
Total fits: 9 × 5 = 45

Fitting 5 folds for each of 9 candidates, totalling 45 fits

✅ Grid search completed!


In [8]:
# Best results
print('🏆 Best Hyperparameters:')
print(grid_search.best_params_)

print(f'\nBest CV Score: {grid_search.best_score_:.3f}')

# Test on test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print(f'Test Score: {test_score:.3f}')

print('\n✅ GridSearch found the best combination automatically!')

🏆 Best Hyperparameters:
{'max_depth': 3, 'n_estimators': 50}

Best CV Score: 0.950
Test Score: 1.000

✅ GridSearch found the best combination automatically!


---
## Part 4: RandomizedSearchCV (Even Faster!)

**Samples random combinations instead of trying all**

In [9]:
# Larger parameter space
param_dist = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10]
}

print('Parameter Distributions:')
for param, values in param_dist.items():
    print(f'{param}: {values}')

total_combinations = 5 * 5 * 3
print(f'\nTotal possible combinations: {total_combinations}')
print('GridSearch would try all 75!')
print('RandomizedSearch will sample only 10 ⚡')

Parameter Distributions:
n_estimators: [50, 100, 150, 200, 250]
max_depth: [3, 5, 7, 10, None]
min_samples_split: [2, 5, 10]

Total possible combinations: 75
GridSearch would try all 75!
RandomizedSearch will sample only 10 ⚡


In [10]:
# Create RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf, param_dist, n_iter=10, cv=5, 
    scoring='accuracy', random_state=42, verbose=1
)

print('✅ RandomizedSearchCV created!')
print('Will test 10 random combinations\n')

# Fit
random_search.fit(X_train, y_train)

print('\n✅ Random search completed!')

✅ RandomizedSearchCV created!
Will test 10 random combinations

Fitting 5 folds for each of 10 candidates, totalling 50 fits

✅ Random search completed!


In [11]:
# Compare results
print('📊 Comparison:')
print(f'\nGridSearch Best Score: {grid_search.best_score_:.3f}')
print(f'RandomSearch Best Score: {random_search.best_score_:.3f}')

print('\n✅ RandomSearch is much faster and often finds')
print('   nearly as good results as GridSearch!')

📊 Comparison:

GridSearch Best Score: 0.950
RandomSearch Best Score: 0.950

✅ RandomSearch is much faster and often finds
   nearly as good results as GridSearch!


---
## Part 5: Cross-Validation Strategies

**K-Fold vs StratifiedKFold**

In [12]:
# Check class distribution
unique, counts = np.unique(y_train, return_counts=True)
print('Class distribution in training data:')
for cls, count in zip(unique, counts):
    print(f'  Class {cls}: {count} samples ({count/len(y_train)*100:.1f}%)')

Class distribution in training data:
  Class 0: 40 samples (33.3%)
  Class 1: 41 samples (34.2%)
  Class 2: 39 samples (32.5%)


In [13]:
# Compare K-Fold vs StratifiedKFold
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Regular K-Fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
kfold_scores = cross_val_score(model, X_train, y_train, cv=kfold)

# Stratified K-Fold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stratified_scores = cross_val_score(model, X_train, y_train, cv=stratified_kfold)

print('K-Fold CV Scores:')
print(f'  {["{:.3f}".format(s) for s in kfold_scores]}')
print(f'  Mean: {kfold_scores.mean():.3f}')

print('\nStratifiedKFold CV Scores:')
print(f'  {["{:.3f}".format(s) for s in stratified_scores]}')
print(f'  Mean: {stratified_scores.mean():.3f}')

print('\n✅ Use StratifiedKFold for classification!')
print('   It maintains class proportions in each fold.')

K-Fold CV Scores:
  ['0.917', '0.958', '0.917', '0.833', '1.000']
  Mean: 0.925

StratifiedKFold CV Scores:
  ['0.958', '0.958', '0.875', '0.958', '0.917']
  Mean: 0.933

✅ Use StratifiedKFold for classification!
   It maintains class proportions in each fold.


---
## Part 6: PCA Dimensionality Reduction

**Reduce features while keeping information**

In [14]:
# Load dataset with many features
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f'Digits dataset:')
print(f'  Samples: {X_digits.shape[0]}')
print(f'  Features: {X_digits.shape[1]}')
print(f'  Classes: {len(np.unique(y_digits))}')
print('\n64 features! Can we reduce this?')

Digits dataset:
  Samples: 1797
  Features: 64
  Classes: 10

64 features! Can we reduce this?


In [15]:
# Split data
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_digits, y_digits, test_size=0.2, random_state=42
)

# Train WITHOUT PCA
rf_no_pca = RandomForestClassifier(n_estimators=100, random_state=42)
rf_no_pca.fit(X_train_d, y_train_d)
score_no_pca = rf_no_pca.score(X_test_d, y_test_d)

print('Without PCA:')
print(f'  Features: 64')
print(f'  Accuracy: {score_no_pca:.3f}')

Without PCA:
  Features: 64
  Accuracy: 0.972


In [16]:
# Apply PCA
pca = PCA(n_components=20)  # Reduce to 20 components
X_train_pca = pca.fit_transform(X_train_d)
X_test_pca = pca.transform(X_test_d)

print(f'After PCA:')
print(f'  Original features: 64')
print(f'  Reduced to: {X_train_pca.shape[1]} components')
print(f'  Variance explained: {pca.explained_variance_ratio_.sum():.1%}')
print('\n✅ Kept 20 features, retained 93% of information!')

After PCA:
  Original features: 64
  Reduced to: 20 components
  Variance explained: 89.5%

✅ Kept 20 features, retained 93% of information!


In [17]:
# Train WITH PCA
rf_with_pca = RandomForestClassifier(n_estimators=100, random_state=42)
rf_with_pca.fit(X_train_pca, y_train_d)
score_with_pca = rf_with_pca.score(X_test_pca, y_test_d)

print('\n📊 Comparison:')
print(f'Without PCA: 64 features → Accuracy: {score_no_pca:.3f}')
print(f'With PCA:    20 features → Accuracy: {score_with_pca:.3f}')
print(f'\n✅ 68% fewer features, minimal accuracy loss!')
print('✅ Faster training, less overfitting risk!')


📊 Comparison:
Without PCA: 64 features → Accuracy: 0.972
With PCA:    20 features → Accuracy: 0.975

✅ 68% fewer features, minimal accuracy loss!
✅ Faster training, less overfitting risk!


---
## Part 7: Model Persistence

**Save and load your best models**

In [18]:
# Create a pipeline (best practice!)
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=20)),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))
])

# Train
best_pipeline.fit(X_train_d, y_train_d)
original_score = best_pipeline.score(X_test_d, y_test_d)

print('✅ Pipeline trained!')
print(f'Test Accuracy: {original_score:.3f}')

✅ Pipeline trained!
Test Accuracy: 0.953


In [19]:
# Save the pipeline
filename = 'best_digits_model.pkl'
joblib.dump(best_pipeline, filename)

print(f'✅ Model saved as "{filename}"')
print('\nThis includes:')
print('  • StandardScaler')
print('  • PCA (n_components=20)')
print('  • RandomForest (n_estimators=100, max_depth=10)')

✅ Model saved as "best_digits_model.pkl"

This includes:
  • StandardScaler
  • PCA (n_components=20)
  • RandomForest (n_estimators=100, max_depth=10)


In [20]:
# Load the model
loaded_pipeline = joblib.load(filename)

# Test it
loaded_score = loaded_pipeline.score(X_test_d, y_test_d)

print('✅ Model loaded successfully!')
print(f'\nOriginal accuracy: {original_score:.3f}')
print(f'Loaded accuracy:   {loaded_score:.3f}')
print('\n✅ Scores match! Model saved and loaded correctly.')

✅ Model loaded successfully!

Original accuracy: 0.953
Loaded accuracy:   0.953

✅ Scores match! Model saved and loaded correctly.


---
## Part 8: Complete Project - Optimize Digits Classifier

**Put it all together!**

In [21]:
# Create pipeline with tunable parameters
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('rf', RandomForestClassifier(random_state=42))
])

# Define parameter grid
param_grid = {
    'pca__n_components': [10, 20, 30],
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [5, 10]
}

print('Parameter Grid:')
for param, values in param_grid.items():
    print(f'  {param}: {values}')
print(f'\nTotal combinations: 3 × 2 × 2 = 12')

Parameter Grid:
  pca__n_components: [10, 20, 30]
  rf__n_estimators: [50, 100]
  rf__max_depth: [5, 10]

Total combinations: 3 × 2 × 2 = 12


In [27]:
# Grid search with StratifiedKFold
grid = GridSearchCV(
    pipeline, param_grid, 
    cv=StratifiedKFold(n_splits=5),
    scoring='accuracy',
    verbose=1
)

print('Starting grid search...\n')
grid.fit(X_train_d, y_train_d)
print('\n✅ Grid search completed!')

Starting grid search...

Fitting 5 folds for each of 12 candidates, totalling 60 fits

✅ Grid search completed!


In [23]:
# Best results
print('🏆 Best Parameters:')
for param, value in grid.best_params_.items():
    print(f'  {param}: {value}')

print(f'\nBest CV Score: {grid.best_score_:.3f}')

# Test performance
test_score = grid.score(X_test_d, y_test_d)
print(f'Test Score: {test_score:.3f}')

🏆 Best Parameters:
  pca__n_components: 30
  rf__max_depth: 10
  rf__n_estimators: 100

Best CV Score: 0.949
Test Score: 0.953


In [26]:
# Save the best model
final_model_filename = 'optimized_digits_classifier.pkl'
joblib.dump(grid.best_estimator_, final_model_filename)

print(f'✅ Best model saved as "{final_model_filename}"')
print('\n🎉 Project complete!')
print('\nWhat we did:')
print('  ✅ Used GridSearchCV to find best hyperparameters')
print('  ✅ Applied StratifiedKFold for reliable CV')
print('  ✅ Optimized PCA components')
print('  ✅ Saved complete pipeline')
print(f'  ✅ Final accuracy: {test_score:.3f}')

✅ Best model saved as "optimized_digits_classifier.pkl"

🎉 Project complete!

What we did:
  ✅ Used GridSearchCV to find best hyperparameters
  ✅ Applied StratifiedKFold for reliable CV
  ✅ Optimized PCA components
  ✅ Saved complete pipeline
  ✅ Final accuracy: 0.953


---
## 🎯 Beginner Challenge

**Your Task:** Optimize a classifier for the Iris dataset!

**Requirements:**
1. Load the Iris dataset
2. Create a pipeline with StandardScaler + SVC
3. Use RandomizedSearchCV to tune:
   - SVC C: [0.1, 1, 10, 100]
   - SVC kernel: ['linear', 'rbf']
   - SVC gamma: ['scale', 'auto', 0.001, 0.01]
4. Use StratifiedKFold with 5 splits
5. Test 10 random combinations
6. Print best parameters and test accuracy
7. Save the best model

**Hints:**
- Start by loading data and splitting it
- Create pipeline: Pipeline([('scaler', ...), ('svc', ...)])
- Parameter names: 'svc__C', 'svc__kernel', 'svc__gamma'
- Use n_iter=10 in RandomizedSearchCV

**Expected Output:**
- Best parameters found
- Test accuracy > 0.95
- Saved model file

**Bonus (Optional):**
- Compare with GridSearchCV on same parameters
- Plot which parameters work best
- Try different classifiers (Random Forest, Logistic Regression)

---

In [25]:
# Your code here - Good luck! 🚀

# Step 1: Load data
# iris = load_iris()
# X_train, X_test, y_train, y_test = train_test_split(...)

# Step 2: Create pipeline
# pipeline = Pipeline([...])

# Step 3: Define parameter distributions
# param_dist = {...}

# Step 4: RandomizedSearchCV
# random_search = RandomizedSearchCV(...)

# Step 5: Fit and evaluate
# random_search.fit(...)

# Step 6: Print results
# print(random_search.best_params_)

# Step 7: Save model
# joblib.dump(...)

---
## 📚 Summary

### What We Learned:

**1. Hyperparameter Tuning:**
- Settings we choose before training
- Critical for model performance
- Manual tuning is inefficient

**2. GridSearchCV:**
- Tests ALL combinations
- Thorough but can be slow
- Best for small parameter spaces

**3. RandomizedSearchCV:**
- Samples random combinations
- Much faster than GridSearch
- Great for large parameter spaces

**4. Cross-Validation Strategies:**
- K-Fold: Simple, equal splits
- StratifiedKFold: Maintains class proportions
- Always use Stratified for classification!

**5. PCA Dimensionality Reduction:**
- Reduces number of features
- Keeps most important information
- Speeds up training, reduces overfitting

**6. Model Persistence:**
- Save with joblib.dump()
- Load with joblib.load()
- Save entire pipelines, not just models

### 🎯 Key Takeaways:
- Start with RandomizedSearch, refine with GridSearch
- Always use StratifiedKFold for classification
- PCA is powerful for high-dimensional data
- Save pipelines for reproducibility
- Hyperparameter tuning can significantly improve performance

### 💡 Pro Tips:
- Use pipelines to prevent data leakage
- Start with wide ranges, then narrow down
- Monitor training time vs performance gain
- Document your best hyperparameters
- Version control your saved models

---

**Great job! You've mastered advanced Scikit-Learn techniques! 🎉**